# Jacobian Transpose Inverse Kinematics and Control

In this notebook, we'll demonstrate an extremely simple approach for computing basic inverse kinematics (IK) and controlling the position of some point on our robot using the Jacobian transpose. 

In [1]:
using RigidBodyDynamics
using RigidBodyDynamics.PDControl
using StaticArrays

First we'll load our double pendulum robot from URDF:

In [2]:
urdf = Pkg.dir("RigidBodyDynamics", "test", "urdf", "Acrobot.urdf")
mechanism = parse_urdf(Float64, urdf)
state = MechanismState(mechanism)
mechanism

Spanning tree:
Vertex: world (root)
  Vertex: base_link, Edge: base_link_to_world
    Vertex: upper_link, Edge: shoulder
      Vertex: lower_link, Edge: elbow
No non-tree joints.

Now we choose a point on the robot to control. We'll pick the end of the second link, which is located 2m from the origin of the `lower_link` body:

In [3]:
body = findbody(mechanism, "lower_link")
point = Point3D(default_frame(body), 0., 0, -2)

Point3D in "after_elbow": [0.0, 0.0, -2.0]

Let's visualize the mechanism and its attached point. For visualization, we'll use [MeshCatMechanisms.jl](https://github.com/JuliaRobotics/MeshCatMechanisms.jl) with [Blink.jl](https://github.com/JunoLab/Blink.jl). 

(Note: the `#NBSKIP` comments are to skip these cells during `Pkg.test("RigidBodyDynamics")`)

In [4]:
#NBSKIP
using MeshCatMechanisms
using Blink

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='pkg/WebIO/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='pkg/WebIO/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...


In [5]:
#NBSKIP
# Create the visualizer
vis = MechanismVisualizer(mechanism, URDFVisuals(urdf))
# Render our target point attached to the robot as a sphere with radius 0.07
setelement!(vis, point, 0.07)
# Open the visualizer in a new Blink window
open(vis, Window())

Blink.AtomShell.Window(1, Blink.AtomShell.Electron(Process(`/home/rdeits/.julia/v0.6/Blink/deps/atom/electron /home/rdeits/.julia/v0.6/Blink/src/AtomShell/main.js port 2857`, ProcessRunning), TCPSocket(RawFD(54) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#3))), Blink.Page(1, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(58) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 1, Nullable{Any}(true))))

## Inverse Kinematics

First, let's use the point jacobian to solve a simple inverse kinematics problem. Given a target location `desired` expressed in world frame, we want to find the joint angles `q` such that the `point` attached to the robot is at the desired location. 

To do that, we'll iteratively update `q` by applying: 

\begin{align}
\Delta q = \alpha \, J_p^\top \, \Delta p
\end{align}

where $\alpha$ is our step size (equivalent to a learning rate in gradient descent) and $\Delta p$ is the error in the position of our target point. 

In [6]:
function jacobian_transpose_ik!(state::MechanismState,
                               body::RigidBody,
                               point::Point3D,
                               desired::Point3D;
                               α=0.1,
                               iterations=100)
    mechanism = state.mechanism
    world = root_frame(mechanism)
    
    # Compute the joint path from world to our target body
    p = path(mechanism, root_body(mechanism), body)
    # Allocate the point jacobian (we'll update this in-place later)
    Jp = point_jacobian(state, p, transform(state, point, world))
    
    q = copy(configuration(state))
    
    for i in 1:iterations
        # Update the position of the point
        point_in_world = transform(state, point, world)
        # Update the point's jacobian
        point_jacobian!(Jp, state, p, point_in_world)
        # Compute an update in joint coordinates using the jacobian transpose
        Δq = α * Array(Jp)' * (transform(state, desired, world) - point_in_world).v
        # Apply the update
        q .= configuration(state) .+ Δq
        set_configuration!(state, q)
    end
    state
end

jacobian_transpose_ik! (generic function with 1 method)

To use our IK method, we just have to set our current state and choose a desired location for the tip of the robot's arm: 

In [7]:
rand!(state)

true

In [8]:
#NBSKIP
set_configuration!(vis, configuration(state))

In [9]:
# Choose a desired location. We'll move the tip of the arm to
# [1, 0, 2]
desired = Point3D(root_frame(mechanism), 1., 0, 2)
# Run the IK, updating `state` in place
jacobian_transpose_ik!(state, body, point, desired)

MechanismState{Float64, Float64, Float64, …}(…)

In [10]:
#NBSKIP
set_configuration!(vis, configuration(state))

In [11]:
# We asked for our point to be close to [1, 0, 2], 
# but since the arm cannot move in the y direction at all
# we end up near [1, 0.25, 2] instead
transform(state, point, root_frame(mechanism))

Point3D in "world": [0.997375, 0.25, 1.99007]

We can try varying the target and watching the IK solution change:

In [12]:
qs = typeof(configuration(state))[]

# Vary the desired x position from -1 to 1
for x in linspace(-1, 1, 100)
    desired = Point3D(root_frame(mechanism), x, 0, 2)
    jacobian_transpose_ik!(state, body, point, desired)
    push!(qs, copy(configuration(state)))
end

In [13]:
#NBSKIP
ts = collect(linspace(0, 1, length(qs)))
setanimation!(vis, ts, qs)

true

## Control

Now let's use the same principle to generate torques and actually control the robot. To make things more interesting, let's get the end of the robot's arm to trace out a circle. 

In [14]:
circle_origin = SVector(0., 0.25, 2)
radius = 0.5
ω = 1.0  # radians per second at which the point should move in its circle

1.0

In [15]:
#NBSKIP
using MeshCat
using GeometryTypes: Point

In [16]:
#NBSKIP

# Draw the circle in the viewer
θ = repeat(linspace(0, 2π, 100), inner=(2,))[2:end]
cx, cy, cz = circle_origin
geometry = PointCloud(Point.(cx .+ radius .* sin.(θ), cy, cz .+ 0.5 .* cos.(θ)))
setobject!(vis[:circle], LineSegments(geometry, LineBasicMaterial()))

MeshCat Visualizer with path /meshcat/circle

In [17]:
rand!(state)

true

In [18]:
typeof(body)

RigidBodyDynamics.RigidBody{Float64}

In [19]:
typeof(point)

RigidBodyDynamics.Spatial.Point3D{StaticArrays.SArray{Tuple{3},Float64,1,3}}

In [20]:
typeof(path(mechanism, root_body(mechanism), body))

RigidBodyDynamics.Graphs.TreePath{RigidBodyDynamics.RigidBody{Float64},RigidBodyDynamics.Joint{Float64,JT} where JT<:RigidBodyDynamics.JointType{Float64}}

In [21]:
typeof(point_jacobian(state, path(mechanism, root_body(mechanism), body), point))

RigidBodyDynamics.Spatial.PointJacobian{Array{Float64,2}}

In [22]:
struct PointCircleController{P <: RigidBodyDynamics.TreePath, V <: AbstractVector}
    body::RigidBody{Float64}
    point::Point3D{SVector{3, Float64}}
    circle_origin::SVector{3, Float64}
    radius::Float64
    ω::Float64
    path::P
    Jp::PointJacobian{Matrix{Float64}}
    v̇::V
end

In [28]:
p = path(mechanism, root_body(mechanism), body)
controller = PointCircleController(
    body,
    point,
    circle_origin,
    radius,
    ω,
    p,
    point_jacobian(state, p, transform(state, point, root_frame(mechanism))),
    similar(velocity(state))
)

PointCircleController{RigidBodyDynamics.Graphs.TreePath{RigidBodyDynamics.RigidBody{Float64},RigidBodyDynamics.Joint{Float64,JT} where JT<:RigidBodyDynamics.JointType{Float64}},RigidBodyDynamics.CustomCollections.SegmentedVector{RigidBodyDynamics.JointID,Float64,Base.OneTo{RigidBodyDynamics.JointID},Array{Float64,1}}}(RigidBody: "lower_link", Point3D in "after_elbow": [0.0, 0.0, -2.0], [0.0, 0.25, 2.0], 0.5, 1.0, Path from world to lower_link:
↓ base_link_to_world
↓ shoulder
↓ elbow
, RigidBodyDynamics.Spatial.PointJacobian{Array{Float64,2}}([-2.41087 -1.41834; 0.0 0.0; -1.5321 -1.41008], CartesianFrame3D: "world" (id = 0)), [6.95072e-310, 0.0])

In [42]:
function (c::PointCircleController)(τ, t, state)
    mechanism = state.mechanism
    world = root_frame(mechanism)
    desired = Point3D(world, 
                      c.circle_origin .+ c.radius .* SVector(sin(t / c.ω), 0, cos(t / c.ω)))
    point_in_world = transform_to_root(state, c.body) * c.point
    point_jacobian!(c.Jp, state, c.path, point_in_world)
    Kp = 200
    Kd = 20
    Δp = desired - point_in_world
    c.v̇ .= Kp * Array(c.Jp)' * Δp.v .- 20 .* velocity(state)
    τ .= inverse_dynamics(state, c.v̇)
end

In [43]:
ts, qs, vs = simulate(state, 10, controller);

In [44]:
setanimation!(vis, ts, qs)

true

In [101]:
gains = RigidBodyDynamics.PDControl.PDGains(10, 1)
H = transform_to_root(state, effector_body)
T = transform(twist_wrt_world(state, effector_body), inv(H))
a = PDControl.pd(gains, translation(H) - [1, 0, 1], linear(T))

Ṫ_desired = SpatialAcceleration(T.body, T.base, T.frame, zero(a), a)

SpatialAcceleration of "after_elbow" w.r.t "world" in "after_elbow":
angular: [0.0, 0.0, 0.0], linear: [8.81691, -2.5, -0.0578644]

In [102]:
function point_acceleration(state, v̇, body)
    p = path(state.mechanism, root_body(state.mechanism), body)
    J = geometric_jacobian(state, p)
    J̇v = -bias_acceleration(state, source(p)) + bias_acceleration(state, target(p))
    SpatialAcceleration(J, v̇) + J̇v
end

point_acceleration (generic function with 1 method)

In [103]:
transform(state,
          -point_acceleration(state, [0, 1], effector_body),
          Ṫ_desired.frame) + Ṫ_desired

SpatialAcceleration of "after_elbow" w.r.t "after_elbow" in "after_elbow":
angular: [0.0, -1.0, 0.0], linear: [8.83602, -2.5, -0.0555152]

In [104]:
using JuMP
using OSQP

In [134]:
set_configuration!(state, randn(2))
set_velocity!(state, randn(2))

true

In [133]:
function controller!(τ::AbstractVector, state::MechanismState, body, target=[-1, 0, -1])
    solver = OSQPMathProgBaseInterface.OSQPSolver(eps_abs = 1e-8, eps_rel = 1e-16, max_iter = 10000)
    MathProgBase.setparameters!(solver, Silent = true)
    model = JuMP.Model(solver=solver)
    @variable model v̇[1:2]
    obj = dot(v̇, v̇)
    
    gains = RigidBodyDynamics.PDControl.PDGains(1, 0.5)
    H = transform_to_root(state, body)
    T = transform(twist_wrt_world(state, body), inv(H))
    a = PDControl.pd(gains, translation(H) - target, linear(T))

    Ṫ_desired = SpatialAcceleration(T.body, T.base, T.frame, zero(a), a)

    error = transform(state,
              -point_acceleration(state, v̇, body),
              Ṫ_desired.frame) + Ṫ_desired
    obj += 10 * dot(error.linear, error.linear)
    @objective model Min obj
    solve(model)
    v̇_out = similar(velocity(state))
    v̇_out .= getvalue(v̇)
    τ .= inverse_dynamics(state, v̇_out)
end
    

controller! (generic function with 4 methods)

In [136]:
ts, qs, vs = simulate(state, 
    5, 
    (τ, t, state) -> controller!(τ, state, effector_body),
    Δt=1e-3);

setanimation!(vis, ts, qs)

true